# Testing of urllib to import nhd files into memory and push into ScienceBase.  This method first creates a SB item, then attemps to attach files.

In [1]:
#Fails at Shapefile (The shapefile components may need to be all uploaded at once?)

import urllib
import io
from zipfile import ZipFile
import pysb
import getpass
import time


#SB Login
sb = pysb.SbSession()
sb.loginc('dwieferich@usgs.gov')
time.sleep(2)   #suggestion from SB, to allow time for login



········


In [2]:
#Create SB item 
new_item = {'title': 'NHDPlusV1 SourisRedRainy Files Test',
    'parentId': sb.get_my_items_id(),
    'provenance': {'annotation': 'Python ScienceBase REST test script'}}
#new_item = sb.create_item(new_item)
print("NEW ITEM: " + str(new_item))



NEW ITEM: {'title': 'NHDPlusV1 SourisRedRainy Files Test', 'parentId': '53ece718e4b02bf5a7681838', 'provenance': {'annotation': 'Python ScienceBase REST test script'}}


In [ ]:
#Loop through files from NHDPlusV1 ftp, first read to memory, then upload file to SB item
flist = ['NHDPlus09/NHDFlowlineVAA.dbf','NHDPlus09/Hydrography/NHDFlowline.dbf','NHDPlus09/Hydrography/NHDFlowline.prj','NHDPlus09/Hydrography/nhdflowline.shp','NHDPlus09/Hydrography/nhdflowline.shx']    
mysock = urllib.request.urlopen('ftp://ftp.horizon-systems.com/NHDplus/NHDPlusV1/SourisRedRainy/NHDPlus09V01_02_NHD.zip')  


In [ ]:
memfile = io.BytesIO(mysock.read())
with ZipFile(memfile, 'r') as ftpl:
    for file in flist:
        test = ftpl.extract(file)
        new_item = sb.upload_file_to_item(new_item, test, scrape_file=False)
ZipFile.close(ftpl)

In [ ]:
UpdateItem = {'id': new_item,
              'title': 'NHDPlusV1 Processing Region 9; Files Used in the Biogeographic Information System',
              'body': 'A subset of files from within processing region 9 of the NHDPlus Version 1.  Although reorganized, the files within the attachments are unaltered from the NHDPlus Version 1 (see acquisition date listed within this metadata). This item links to python code used to generate the item.',
              'purpose': 'This item is intended to preseve specific versions of files being used in the Biogeographic Information System.',
              'dates': [{'type': 'Acquisition', 'dateString': AcqDate, 'label': 'Acquisition'}],
              'webLinks': [{"type":"sourceCode","typeLabel":"Source Code","uri":"https://github.com/dwief-usgs/BCB_Ipython_Notebooks/blob/master/Reg9_NHDPlusV21_IntoSB_BIS.ipynb","rel":"related","title":"Python Code Used to Develop and Populate This SB Item","hidden":False},{"type":"webLink","typeLabel":"Web Link","uri":"http://www.horizon-systems.com/NHDPlus/NHDPlusV1_home.php","rel":"related","title":"Additional Information About the NHDPlusV1","hidden":False}],
              'contacts': [{"name":"Horizon Systems","type":"Data Owner","contactType":"organization","onlineResource":"http://www.horizon-systems.com","organization":{},"primaryLocation":{"streetAddress":{},"mailAddress":{}}},{"name":"Daniel J Wieferich","oldPartyId":66431,"type":"Contact","contactType":"person","email":"dwieferich@usgs.gov","active":True,"jobTitle":"Physical Scientist","firstName":"Daniel","middleName":"J","lastName":"Wieferich","organization":{"displayText":"Biogeographic Characterization"},"primaryLocation":{"name":"CN=Daniel J Wieferich,OU=CSS,OU=Users,OU=OITS,OU=DI,DC=gs,DC=doi,DC=net - Primary Location","building":"DFC Bldg 810","buildingCode":"KBT","officePhone":"3032024594","faxPhone":"3032024710","streetAddress":{"line1":"W 6th Ave Kipling St","city":"Lakewood","state":"CO","zip":"80225"},"mailAddress":{}},"orcId":"0000-0003-1554-7992"}],
              "tags":[{"type":"Theme","scheme":"BIS","name":"NHDPlusV2.1"},{"type":"Theme","scheme":"BIS","name":"Reg9"}]
             }

updateItem = sb.updateSbItem(UpdateItem)